In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
preprocessor = mz.models.ESIM.get_default_preprocessor()

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8763.18it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5453.94it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3008929.55it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10463.76it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5391.61it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 1001557.40it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 10261.40it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [5]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f29081202b0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f28a032d400>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [8]:
padding_callback = mz.models.ESIM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    shuffle=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    shuffle=False,
    callback=padding_callback
)

In [9]:
model = mz.models.ESIM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['mask_value'] = 0
model.params['dropout'] = 0.2
model.params['hidden_size'] = 200
model.params['lstm_layer'] = 1

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ESIM(
  (embedding): Embedding(30058, 300)
  (rnn_dropout): RNNDropout(p=0.2, inplace=False)
  (input_encoding): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 100, bidirectional=True)
    )
  )
  (attention): BidirectionalAttention()
  (projection): Sequential(
    (0): Linear(in_features=800, out_features=200, bias=True)
    (1): ReLU()
  )
  (composition): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(200, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=800, out_features=200, bias=True)
    (2): Tanh()
    (3): Dropout(p=0.2, inplace=False)
  )
  (out): Linear(in_features=200, out_features=1, bias=True)
)
Trainable params:  9901201


In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-255 Loss-1.824]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5725 - normalized_discounted_cumulative_gain@5(0.0): 0.6183 - mean_average_precision(0.0): 0.5764



[Iter-510 Loss-0.494]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5458 - normalized_discounted_cumulative_gain@5(0.0): 0.6114 - mean_average_precision(0.0): 0.5602



[Iter-765 Loss-0.111]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5442 - normalized_discounted_cumulative_gain@5(0.0): 0.589 - mean_average_precision(0.0): 0.5456



[Iter-1020 Loss-0.060]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5198 - normalized_discounted_cumulative_gain@5(0.0): 0.5767 - mean_average_precision(0.0): 0.537



[Iter-1275 Loss-0.056]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5136 - normalized_discounted_cumulative_gain@5(0.0): 0.5721 - mean_average_precision(0.0): 0.5278



[Iter-1530 Loss-0.039]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5324 - normalized_discounted_cumulative_gain@5(0.0): 0.5721 - mean_average_precision(0.0): 0.5429



[Iter-1785 Loss-0.017]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5197 - normalized_discounted_cumulative_gain@5(0.0): 0.5838 - mean_average_precision(0.0): 0.54



[Iter-2040 Loss-0.020]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5271 - normalized_discounted_cumulative_gain@5(0.0): 0.5819 - mean_average_precision(0.0): 0.5432



[Iter-2295 Loss-0.018]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5122 - normalized_discounted_cumulative_gain@5(0.0): 0.5657 - mean_average_precision(0.0): 0.5277



[Iter-2550 Loss-0.018]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5214 - normalized_discounted_cumulative_gain@5(0.0): 0.5742 - mean_average_precision(0.0): 0.5381

Cost time: 9662.94213104248s
